# Plate Mapper

<!-- ![Sexton Labs](images/sextonlabs.png =336x410)
 -->
<img src="images/sextonlabs.png" width="20%">

## Instructions

1. run first cell and selected inputs `csv` and output directory
2. Fill out UI elements for compound
3. Hit the go button
4. Repeat for other compounds
5. Run other cells, I'll add a button when I care
6. Change save name for csv and run cell

In [ ]:
import platemapper as mapper
import pandas as pd
import re # REEEEE

from IPython.display import *
import ipywidgets as widgets
from ipywidgets import *

import seaborn as sns
import matplotlib as plt


sfc = mapper.make_chooser('Select source csv', False)
fc = mapper.make_chooser('output directory')
display(VBox([sfc,fc]))

In [ ]:
source = sfc.selected
out = fc.selected_path
plate = mapper.Plate(source, out)
display(VBox(plate.get_elements()))

In [ ]:
plate.gen_plate()
plate.save_plate()